In [2]:
%run "/home/jovyan/work/workspace/functions/create_spark_session.ipynb"

In [3]:
%run "/home/jovyan/work/workspace/functions/paths.ipynb"

In [4]:
%run "/home/jovyan/work/workspace/functions/dates.ipynb"

In [11]:
from pyspark.sql.functions import split,col,explode, upper, trim 

In [12]:
df = spark.read.format('delta').load(f'{silver}channel_group')

In [13]:
df.show(truncate=False)

+------------------------+--------------+-------------+----------+
|tpcanal                 |tpgrupo       |tptransacao  |dtcarga   |
+------------------------+--------------+-------------+----------+
|COLLEGE/UNIVERSITY      |ACADEMIC      |NON ALCOHOLIC|2026-02-14|
|ALL OTHER               |OTHER         |MIX          |2026-02-14|
|RESTAURANT              |ENTERTAINMENT |ALCOHOLIC    |2026-02-14|
|MILITARY-EXCHANGE       |GOV & MILITARY|MIX          |2026-02-14|
|SPORT VENUE             |ENTERTAINMENT |ALCOHOLIC    |2026-02-14|
|PRIMARY/SECONDARY SCHOOL|ACADEMIC      |NON ALCOHOLIC|2026-02-14|
|MILITARY-OTHER          |GOV & MILITARY|MIX          |2026-02-14|
|WHOLESALE               |GROCERY       |MIX          |2026-02-14|
|GOVERNMENT(NON-MILITARY)|GOV & MILITARY|NON ALCOHOLIC|2026-02-14|
|PLANT / OFFICE          |SERVICES      |MIX          |2026-02-14|
|GENERAL MERCHANDISER    |GROCERY       |MIX          |2026-02-14|
|SUPERMARKET             |GROCERY       |MIX          |2026-02

In [15]:
df = (

    df.withColumn('split', split(col('tpcanal'),'/'))
    .withColumn('tpcanal', explode(col('split')))
    .withColumn('tpcanal', trim(col('tpcanal')))    
    .drop('split')
)

In [18]:
df.write.partitionBy('dtcarga').format('delta').mode('overwrite').save(f'{gold}channel_group')